# Reading Checkpoint

In [11]:
!pip install tensorflow_hub


In [12]:
!pip install keras tf-models-official pydot graphviz

In [13]:
import os

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

from keras.utils import np_utils

import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization as tokenization

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt


In [14]:
from google.colab import drive

drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [15]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {`s`
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [16]:
from tensorflow.keras.models import load_model
from tensorflow import keras

In [17]:
path = "/content/gdrive/My Drive/checkpoint" 
optimizer = nlp.optimization.create_optimizer(
    2e-5, num_train_steps=417
, num_warmup_steps=41)
model = keras.models.load_model(path, custom_objects={'AdamWeightDecay': optimizer})

In [18]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2",
                            trainable=True)

In [19]:

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)


In [20]:
def encode_names(n, tokenizer):
   tokens = list(tokenizer.tokenize(n))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

def bert_encode(string_list, tokenizer, max_seq_length):
  num_examples = len(string_list)
  
  string_tokens = tf.ragged.constant([
      encode_names(n, tokenizer) for n in np.array(string_list)])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*string_tokens.shape[0]
  input_word_ids = tf.concat([cls, string_tokens], axis=-1)

  input_mask = tf.ones_like(input_word_ids).to_tensor(shape=(None, max_seq_length))

  type_cls = tf.zeros_like(cls)
  type_tokens = tf.ones_like(string_tokens)
  input_type_ids = tf.concat(
      [type_cls, type_tokens], axis=-1).to_tensor(shape=(None, max_seq_length))

  inputs = {
      'input_word_ids': input_word_ids.to_tensor(shape=(None, max_seq_length)),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

  return inputs

In [21]:
tokenizer

In [22]:
pip install transformers==3


#Face Emotion detection


In [23]:
from keras.models import load_model
from IPython.display import display, Javascript
from google.colab.output import eval_js
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np


In [24]:
face_classifier = cv2.CascadeClassifier('/content/gdrive/My Drive/haarcascade_frontalface_default.xml')
classifier = load_model('/content/gdrive/My Drive/Emotion_Detection.h5')

class_labels = ['Angry','Happy','Neutral','Sad','Surprise']

In [25]:
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [26]:
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [27]:
emotions=[]
emotions

[]

In [28]:
emotions

[]

In [29]:
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]

In [30]:
##################################### mohema tala3ha foo2
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

In [51]:
##################################################################################      Working         ##############################################################################
from collections import Counter
label_html = 'Capturing...'

# initialze bounding box to empty
bbox = ''
count = 0 
def read_face_emotion():
  video_stream()
  emotions=[]
  print('awel read')
  while True:

      js_reply = video_frame(label_html, bbox)
      labels=[]
      if not js_reply:
          break

      # convert JS response to OpenCV Image
      img = js_to_image(js_reply["img"])

      # create transparent overlay for bounding box
      bbox_array = np.zeros([480,640,4], dtype=np.uint8)

      # grayscale image for face detection
      gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

      # get face region coordinates
      faces = face_cascade.detectMultiScale(gray)
      # get face bounding box for overlay
      for (x,y,w,h) in faces:
        bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
        

        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)
            preds = classifier.predict(roi)[0]
           # print("\nprediction = ",preds)
            label=class_labels[preds.argmax()]
          #  print("\nprediction max = ",preds.argmax())
          #  print("\nlabel = ",label)

            emotions.append(label)
            try:
              if(len(emotions)%5==0):
                print('ana gowa')
                return most_frequent(emotions)

              
            except Exception as e:
              continue
               


            

            label_position = (x,y)
            cv2.putText(img,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)

  # return(mode(emotions))

    
      # convert overlay of bbox into bytes
      
      # update bbox so next frame gets new overlay
    

In [ ]:
#print('most common '+read_face_emotion())

In [33]:
# def most_common(List):
#     return(mode(List))
#read_face_emotion()
def threaded_face_emotion():
  read_face_emotion()
  most_common(emotions)
  ################################### faksana

In [34]:
# import threading

# t1=threading.Thread(target=read_face_emotion)
# t1.start()
# print(threading.active_count() )
# print('fierst')

# print('hello')

In [35]:
# video_stream()
  ################################### faksana  ################################### faksana  ################################### faksana  ################################### faksana
# # label for video
# label_html = 'Front Camera'
# # initialze bounding box to empty
# bbox = ''
# count = 0 

# emotions=[]
# ###################################
# def face_emotion():
#   video_stream()
#   print('out')
#   while True:
#     print('in')
#     js_reply = video_frame(label_html, bbox)
#     print('2222222222')
#     if not js_reply:
#         print('not')
#         return
#     labels = []
#     print('1')
#     # convert JS response to OpenCV Image
#     img = js_to_image(js_reply["img"])
#     print('2')
#     # create transparent overlay for bounding box
#     bbox_array = np.zeros([480,640,4], dtype=np.uint8)
#     print('3')
#     # grayscale image for face detection
#     gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     print('4')
#     # get face region coordinates
#     faces = face_cascade.detectMultiScale(gray)
#     #faces = face_classifier.detectMultiScale(gray,1.3,5)


#     print('before loop')

#     # get face bounding box for overlay
#     for (x,y,w,h) in faces:
#         print('in for loop')
#        # cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
#         roi_gray = gray[y:y+h,x:x+w]
#         roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

        
#         if np.sum([roi_gray])!=0:
#             roi = roi_gray.astype('float')/255.0
#             roi = img_to_array(roi)
#             roi = np.expand_dims(roi,axis=0)

#         # make a prediction on the ROI, then lookup the class

#             preds = classifier.predict(roi)[0]
#             print("\nprediction = ",preds)
#             label=class_labels[preds.argmax()]
#             emotions.append(label)
#             fs=True
#             print(fs)
#             print(len(emotions))
#             try:
#               print('in try')
#               print(len(emotions))
#               if len(emotions)%5==0:
#                 print('most common emotion: '+most_common(emotions))
#                 fs=False
#                 emotions.clear()
#             except Exception as e:
#               print('unexpected error in face detection')
#               print("\nprediction max = ",preds.argmax())
#             #print("\nlabel = ",label)
#             label_position = (x,y)
#             cv2.putText(img,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
#         else:
#             cv2.putText(img,'No Face Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
#             print("else")
#     # update bbox so next frame gets new overlay
# face_emotion()
  ################################### faksana  ################################### faksana  ################################### faksana  ################################### faksana

# Q&A model now


In [36]:
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering
import torch
import textwrap
from textblob import TextBlob
import nltk

Load the Pretrained Model for Question Answering Bert

In [37]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
tokenizer1 = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

model1 = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

#AutoTokenizer.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")

In [48]:
wrapper = textwrap.TextWrapper(width=80)
import concurrent.futures


def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
 

    

  

   # print('ba3d el thread')



    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer1.encode(question, answer_text)

    # Report how long the input sequence is.
   # print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer1.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0] * num_seg_a + [1] * num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run  question through the model.
    start_scores, end_scores = model1(torch.tensor([input_ids]),  # The tokens representing our input text.
                                     token_type_ids=torch.tensor(
                                         [segment_ids]))  # The segment IDs to differentiate question from answer_text

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer1.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):

        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]

        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
    if "CLS" in answer:
      print('You: '+ question )
      print('Sorry i dont know')
    else:
      print('a5er questions')
      print('You: '+ question )
      print('Answer : '+ answer )
     # print('oh i sense that you are '+emotion)


In [49]:
def answer_this(question,y=1000):
  print('awel answer')
  if(y<200):
    return
 # question = ['which types of vaccines available to cure covid']
  global Selected_Category
  global inputs
  global model
  global prediction
  question1=[question]
  inputs = bert_encode(string_list=list(question1), 
                     tokenizer=tokenizer, 
                     max_seq_length=63)
  prediction = model.predict(inputs)
  possible_categories=['Comparison','Economic_Effects','Having_COVID','Individual_Response','Nomenclature','Origin','Prevention','Reporting','Societal Effects','Societal Response','Speculation','Symptoms','Testing','Transmission','Treatment']
  for i in range (len(possible_categories)):
    if np.argmax(prediction)== i:

      Selected_Category=possible_categories[i]



  # if np.argmax(prediction)== 0:
  #   Selected_Category='Comparison'
    
  # if np.argmax(prediction)== 1:
  #   Selected_Category='Economic_Effects'
    
  # if np.argmax(prediction)== 2:
  #   Selected_Category='Having_COVID'
    
  # if np.argmax(prediction)== 3:
  #   Selected_Category='Individual_Response'
    
  # if np.argmax(prediction)== 4:
  #   Selected_Category='Nomenclature'
    
  # if np.argmax(prediction)== 5:
  #   Selected_Category='Origin'
    
  # if np.argmax(prediction)== 6:
  #   Selected_Category='Prevention'
    
  # if np.argmax(prediction)== 7:
  #   Selected_Category='Reporting'
    
  # if np.argmax(prediction)== 8:
  #   Selected_Category='Societal Effects'
    
  # if np.argmax(prediction)== 9:
  #   Selected_Category='Societal Response'
    
  # if np.argmax(prediction)== 10:
  #   Selected_Category='Speculation'
    
  # if np.argmax(prediction)== 11:
  #   Selected_Category='Symptoms'
    
  # if np.argmax(prediction)== 12:
  #   Selected_Category='Testing'
    
  # if np.argmax(prediction)== 13:
  #   Selected_Category='Transmission'
    
  # if np.argmax(prediction)== 14:
  #   Selected_Category='Treatment'
    

  
  flag_exception = False

  try:
    with open('/content/gdrive/My Drive/Categories/'+Selected_Category+'.txt') as f:
      x=f.readlines()
    Categorial_Abtract=x
    bert_abstract=''.join(Categorial_Abtract)

    bert_abstract
  except Exception as e:
    print('You: '+ question )
    print(Selected_Category +' Category is still not available yet.')
    return
    flag_exception = True

  if  flag_exception == True:
    print(Selected_Category +' Category is still not available yet.')
    return
  else:
    try:
      
      
      #print(bert_abstract)
      answer_question(question,bert_abstract[:y])
    except Exception as e:
      print ('ops an error occured  '+ str(e))
      answer_this(question,y-100)
      

  

In [40]:
# answer_this('what is covid cure')
# answer_this('and where did covid start')
# answer_this('what is the symptoms?')
# answer_this('what is the incubation period?')
# answer_this('when will corona end?')
answer_this('what are type of vaccines to cure corona?')

You: what are type of vaccines to cure corona?
Answer : mrna vaccines


In [52]:
  import concurrent.futures
  
  #print(read_face_emotion())
  with concurrent.futures.ThreadPoolExecutor() as executor:
      future = executor.submit(read_face_emotion)
      future2=executor.submit(answer_this,'what is covid')
      r=future.result()
      print('i sense '+r)
      

<IPython.core.display.Javascript object>

awel answer
awel read
a5er questions
You: what is covid
Answer : a new disease
ana gowa
i sense Happy


In [42]:
video_stream()

<IPython.core.display.Javascript object>

In [43]:
#
 # what is  covid
#video_stream()
# process=[]
# for _ in range(10)
#   p1=Process(target=threaded_face_emotion)
#   p1.start()

# p2=Process(target=bert_answer)
# p2.start()
# p2.join()
# p2.start()
# p1.start()

# p2.join()
# p1.join()

#face_emotion()
# read_face_emotion()
# t1=threading.Thread(target=face_emotion)
# t2=threading.Thread(target=bert_answer)
# t1.start()

#t2.start()
# time.sleep(500)
#t1.join()
#bert_answer()

  #answer_this('what is covid cure')
# answer_this('and where did covid start')
# answer_this('what is the symptoms?')
# answer_this('what is the incubation period?')
# answer_this('how does corona spread?')




# End of Code

In [44]:
import concurrent.futures

def foo():
  return 2

with concurrent.futures.ThreadPoolExecutor() as executor:
    future = executor.submit(foo)
    return_value = future.result()
    print(return_value)

2
